In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from groq import Groq
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

In [6]:
class GroqLLM:
    def __init__(self, client, model):
        self.client = client
        self.model = model

    def __call__(self, query):
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": str(query)}]
        )
        return response.choices[0].message.content

# Initialize like this
llm = GroqLLM(client, "llama-3.3-70b-versatile")

In [13]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://pubmed.ncbi.nlm.nih.gov/11831734/")
document = loader.load()


In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
document = text_splitter.split_documents(document)

In [15]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
embedding = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-en-v1.5")


/home/anuj/anaconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2025-06-22 21:52:48.015859: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-22 21:52:48.310443: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750609368.434992   22114 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750609368.4

In [16]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(document,embedding)

In [17]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context
<context>
{context}
</context>
Question : {input}""")